# Innehållsförteckning  
1. [Introduktion](#1)  
2. [Kolumner](#2)  
    2.1 [Enkät](#2.1)  
    2.2 [Itemnr](#2.2)  
    2.3 [Frekvens](#2.3)
3. [Tabeller](#3)  
    3.1 [Variabler](#3.1)  
    3.2 [Frekvenstabell](#3.2)  
    3.3 [Enkättexter](#3.3)  
    3.4 [Kodlistor](#3.4)  
    3.5 [Svarstexter](#3.5)  
    3.6 [Kodlistor_stor](#3.6)

***

# <a id = "1">1. Introduktion</a>  
I vad som följer kommer kursiv stil endast användas för att benämna *tabeller*. Fetstil kommer endast användas för att benämna **kolumner**, och värden i en kolumn kommer att vara i stil med (t.ex.) `SLOSHW08`, `Missing`, `142`. Även datatyper (t.ex. `str` eller `float64`) och pythonobjekt (t.ex. listor) kommer att ha denna formatering. Den metadata som redan är dokumenterad enligt Vetenskapsrådets riktlinjer ligger i excelfilen *metadata*. Några av tabellerna i relationsdatabsen innehåller kolumner som inte finns i denna fil. Dessa kolumner skapas i avsnittet [Kolumner](#2). Därefter kommer följande tabeller att skapas:  

|Variabler|Frekvenstabell|Enkättexter|Kodlistor|Svarstexter|Kodlistor_stor|  
|:---:|:---:|---|---|---|---|
|Variabel|V_id|Enkättext|Kodlista|Svarstext|K_id|  
|Enkät|Variabel|Källa|From|From|Kodlista|  
|Beskrivning|Enkät|-|Tom|Tom|Kod|  
|Itemnr|Kod|-|-|-|Svarstext|  
|Enkättext|Frekvens|-|-|-|-|  
|Kodlista|-|-|-|-|-|  
  
  
Relationsdatabasen kommmer att innehålla ytterligare fyra tabeller. Dessa är *Begrepp*, *Begrepp_stor*, *Tidsserie* samt *Tidsserie_stor*. För tillfället finns ingen data till dessa tabeller i *metadata* och därför kommer de inte skapas här. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel('Metadata.xlsx')

För att ta fram *Kodlista_stor* används `missing` i **text**. I *metadata* stavas `Missing` ibland med "M" och ibland med "m"(`missing`). Python läser detta som olika `str`, därför ersätts liten bokstav med stor.

In [3]:
df.replace('missing', 'Missing', inplace = True)

In [4]:
df.columns

Index(['Namn', 'Tidsserie', 'Beskrivning', 'Enkät', 'Dubbelkodning',
       'Enkättext', 'From', 'Tom', 'Kodlista', 'From.1', 'Tom.1', 'Kod',
       'Text', 'From.2', 'Tom.2', 'Koncept 1', 'Koncept 2', 'Instrument 1',
       'Instrument 2', 'Källa', 'SLOSHW06', 'frek', 'SLOSHW08', 'frek.1',
       'SLOSHW10', 'frek.2', 'SLOSHW12', 'frek.3', 'SLOSHW14', 'frek.4',
       'SLOSHW16', 'frek.5', 'SLOSHW18', 'frek.6', 'SLOSHNW06', 'frek.7',
       'SLOSHNW08', 'frek.8', 'SLOSHNW10', 'frek.9', 'SLOSHNW12', 'frek.10',
       'SLOSHNW14', 'frek.11', 'SLOSHNW16', 'frek.12', 'SLOSHNW18', 'frek.13'],
      dtype='object')

*** 
# <a id = "2">2. Kolumner</a>  
De kolumner som måste skapas från *metadata* är **Enkät**, **Itemnr** och **Frekvens**.  

***  
## <a id = "2.1">2.1 Enkät</a>  
Den här kolumnen ska säga vilket år (eller vilken våg) variabeln förekommer i. Samma kolumn ska säga huruvida variabeln finns för den arbetande- eller den icke-arbetande delen av SLOSH. Ett exempel på ett värde i kolumnen är `SLOSHW08`. Här står "W" för "working" ("non-working" förkortas "NW") och "08" för den andra SLOSH-vågen (som ägde rum 2008). Värdena i den här kolumnen är namn på (redan existerande) kolumner i *metadata*. Börja med att samla namnen på working-kolumner respektive nonworking-kolumner (från *metadata*) i två listor. Listorna kallas "WORKING" respektive "NONWORKING".

In [5]:
WORKING = ['SLOSHW06', 'SLOSHW08', 'SLOSHW10', 'SLOSHW12', 'SLOSHW14', 'SLOSHW16', 'SLOSHW18']
NONWORKING = ['SLOSHNW06', 'SLOSHNW08', 'SLOSHNW10', 'SLOSHNW12', 'SLOSHNW14', 'SLOSHNW16', 'SLOSHNW18']

Kolumnerna som står för vågor i SLOSH (alltså t.ex. SLOSHW06) innehåller `NaN` ifall en given variabel inte finns med i vågen. Om variabeln istället *finns* kommer vågkolumnen innehålla en `str`. Varje variabel kan förekomma i högst två kolumner (en SLOSHW och en SLOSHNW). Ifall en given variabel förekommer i *endast en* kolumn, t.ex. **SLOSHW06**, ska variabelns värde i **Enkät** vara `SLOSHW06`.

In [6]:
for w,nw in zip(WORKING, NONWORKING):
    df.loc[(df[w].isna() == False) & (df[nw].isna() == True), ['Enkät']] = w
    df.loc[(df[nw].isna() == False) & (df[w].isna() == True), ['Enkät']] = nw

In [7]:
df['Enkät'].value_counts()

SLOSHW06       1210
SLOSHW08        889
w+n             748
SLOSHW10        669
SLOSHW12        579
SLOSHW14        567
SLOSHW16        517
SLOSHW18        426
SLOSHNW08       307
working         181
SLOSHNW12       160
SLOSHNW14       130
SLOSHNW16       106
SLOSHNW18        75
SLOSHNW10        41
non-working      34
SLOSHNW06        24
Name: Enkät, dtype: int64

Frekvenstabellen visar att det finns tre värden i **Enkät** för vilka tillhörande variabler förekommer i två vågkolumner. Detta är oväntat. Alla variabler som förekommer i två vågor ska egentligen ha **Enkät**-värdet `w+n`. Alltså har vi (någonstans) dokumneterat fel. Tillsvidare får dessa variabler **Enkät**-värdet `oklar`.

In [8]:
df.loc[(df['Enkät'] == 'w+n') | (df['Enkät'] == 'working') | (df['Enkät'] == 'non-working'), ['Enkät']] = 'oklar'

  ***  
## <a id = "2.2">2.2 Itemnr</a>  
Nu passar **Enkät** in i relationsdatabasen (med undantag för de variabler som är `oklara`). **Itemnr** ska innehålla `str` som för närvarande ligger utspridda i vågkolumnerna i *metadata*. Ett värde i kolumnen kan t.ex. vara `A26_b`. Här är ett förenklat schema som visar relationen mellan **Itemnr** och vågkolumnerna i *metadata*:

In [9]:
pd.DataFrame([['A25', 'NaN', 'Nan', 'A25'], ['NaN', 'B23', 'Nan', 'B23'], ['NaN', 'NaN', 'C13', 'C13']], 
             columns = ['SLOSHW06', 'SLOSHNW10', 'SLOSHW16', 'Itemnr'])

,SLOSHW06,SLOSHNW10,SLOSHW16,Itemnr
0,A25,NaN,Nan,A25
1,NaN,B23,Nan,B23
2,NaN,NaN,C13,C13


Först skapas en ny kolumn som får namnet "Itemnr", kolumnen fylls med `NaN`.

In [10]:
df['Itemnr'] = np.nan

Proceduren för att välja ut rätt itemnummer illustreras i exemplet nedan.  
***  
  <span style="color:red">**Exempel**

In [11]:
d = pd.DataFrame([[1,2,3], [4,5,6], [1,8,9]], columns = ['A', 'B', 'C'])
d['test'] = 0

In [12]:
d

,A,B,C,test
0,1,2,3,0
1,4,5,6,0
2,1,8,9,0


In [13]:
d.loc[d['A'] == 1, ['test']] = d['B']
d

,A,B,C,test
0,1,2,3,2
1,4,5,6,0
2,1,8,9,8


<span style="color:red">**Exempel slut**  
***

Loopen nedan går igenom vågkolumnerna i *metadata*. Om en variabel har fått värdet `SLOSHW06` i **Enkät** kommer värdet för samma variabel i vågkolumnen **SLOSHW06** att bli variabelns värde i den nya kolumnen **Itemnr**.

In [14]:
for w, nw in zip(WORKING, NONWORKING):
    df.loc[df['Enkät'] == w, ['Itemnr']] = df[w]
    df.loc[df['Enkät'] == nw, ['Itemnr']] = df[nw]

Varabler med **Enkät** `oklar` ges **Itemr** `oklar`. 

In [15]:
df.loc[df['Enkät'] == 'oklar', ['Itemnr']] = 'oklar'

Gör en `value_counts` för att se att någon frekvenstabell i *metadata* hamnat på fel ställe... Kanske har den förskjutits någon kolumn eller så...

***  
## <a id = "2.3">2.3 Frekvens</a>  
Kolumnen med frekvenser görs på ungefär samma sätt som **Itemnr**. I *metadata* ligger en frekvenskolumn intill varje vågkolumn. Exempelvis åtföljs **SLOSHW06** av en kolumn som rymmer frekvenser för de variabler som förekommer i **SLOSHW06**. De variabler som *inte* förekommer i **SLOSHW06** har värdet `NaN`i samma frekvenskolumn.  
  
Skapa först en ny kolumn med namnet "Frekvens" och fyll den med `NaN`.


In [16]:
df['Frekvens'] = np.nan

Skapa två listor, en för arbetande och en för icke-arbetande, och fyll dessa med namnen på frekvenskolumnerna från *metadata*.

In [17]:
WORKFREK = ['frek.1', 'frek.2', 'frek.3', 'frek.4', 'frek.5', 'frek.6', 'frek.7']
NONWORKFREK = ['frek.8', 'frek.9', 'frek.10', 'frek.11', 'frek.12', 'frek.13']

Loopen nedan går igenom frekvenskolumnerna i *metadata*. De variabler som har frekvenser för högst en våg får detta värde i den nya frekvenskolumnen. De variabler som har frekvenser i två vågor, alltså de som i *metadata* är `w+n` samt ej är dubbelkodade, får tillsvidare `NaN` i den nya frekvenskolumnen. Anledningen är att det i avsnittet [Enkät](#2.2) antyds att vissa variabler är feldokumenterade.

In [18]:
for w, nw in zip(WORKFREK, NONWORKFREK):
    df.loc[(df[w].isna() == False) & (df[nw].isna() == True), ['Frekvens']] = df[w]
    df.loc[(df[nw].isna() == False) & (df[w].isna() == True), ['Frekvens']] = df[nw]

***  
# <a id = "3">3. Tabeller</a>  
I det här avnittet kommer tabellerna (förutom de som rör tidsserier och begrepp, se [introduktionen](#1)) till relationsdatabasen att skapas. Först tabellen *Variabler* (som är störst). Därefter *Frekvenstabeller*, *Enkättexter*, *Kodlistor*, *Svarstexter* och till sist *Kodlistor_stor*. 

***  
## <a id = "3.1">3.1 Variabler</a>  
Den här tabellen ska innehålla kolumnerna **Variabel**, **Enkät**, **Beskrivning**, **Itemnr**, **Enkättext** och **Kodlista**. Samtliga kolumner finns redan i *metadata* (numera `df`), dock kallas **Variabel** där för **Namn** och måste därför döpas om. 

In [19]:
Variabler = df[['Namn', 'Enkät', 'Beskrivning', 'Itemnr', 'Enkättext', 'Kodlista']]
Variabler = Variabler.rename({'Namn': 'Variabel'}, axis = 'columns')

Värden i **Variabel** ska vara unika.

In [20]:
Variabler.drop_duplicates(['Variabel'], inplace = True)

***  
## <a id = "3.2">3.2 Frekvenstabeller</a>  
Kolumnerna är **V_id**, **Variabel**, **Enkät**, **Kod** och **Frekvens**.

In [21]:
Frekvenstabeller = df[['Namn', 'Enkät', 'Kod', 'Frekvens']]
Frekvenstabeller = Frekvenstabeller.rename({'Namn': 'Variabel'}, axis = 'columns')

**V_id** är huvudnyckel i tabellen, därför räcker det att döpa om index.

In [22]:
Frekvenstabeller.index.name = 'V_id'

***  
## <a id = "3.3">3.3 Enkättexter</a>  
Består endast av **Enkättext** och **Källa**. Eventuellt bör **Enkättext** göras till index eftersom den är huvudnyckel i tabellen.

In [24]:
Enkättexter = df[['Enkättext', 'Källa']]
Enkättexter.set_index('Enkättext', inplace = True)

***  
## <a id = "3.4">3.4 Kodlistor</a>  
Består av **Kodlista** (nyckel) samt **From.1** och **Tom.1**. Notera att de sista klolumnerna blivit omdöpta automatiskt vid inläsning av *metadata*. 

In [25]:
Kodlistor = df[['Kodlista', 'From.1', 'Tom.1']]
Kodlistor.set_index('Kodlista', inplace = True)

***  
## <a id = "3.5">3.5 Svarstexter</a>  
Består av **Svarstext** (nyckel) samt **From** och **Tom**. Den första kolumnen heter **Text** i *metadata* och döps därför om. 

In [28]:
Svarstexter = df[['Text', 'From', 'Tom']]
Svarstexter.rename(columns = {'Text':'Svarstext'}, inplace = True)
Svarstexter.set_index('Svarstext', inplace = True)

***  
## <a id = "3.6">3.6 Kodlistor_stor</a>  
Den här tabellen ska innehålla kodlistornas namn i kombination med deras kod och svarstext. Varje sådan kombination ska endast förekomma en gång. Börja med att skära ut kolumnerna **Kodlista**, **Kod** och **Text** ur *metadata*.

In [ ]:
d = df[['Kodlista', 'Kod', 'Text']]  
d.head(5)

I den här mindre tabellen förekommer vissa kodlistor flera gånger. För att identifiera dubletter behövs en slags räknare. Börja med att lägga till en ny kolumn fylld med `0`. Kalla kolumnen för **K_id**.

In [ ]:
d.loc[:, 'K_id'] = 0
d.head(5)

**K_id** ska vara unik för varje kombination. Loopen nedan ökar **K_id** med `1` varje gång den passerar en kombination. En kombination identifieras genom att dess sista rad alltid har svarstexten `Missing`.

In [ ]:
for id in range(1, len(d['K_id'])):
    if d.iloc[id-1, 2] == 'Missing':
        d.iloc[id: , 3] = d.iloc[id: , 3] + 1

Loopen nedan lägger varje kodlista i listan `behåll` endast en gång. Varje gång en kodlista läggs i `behåll` lägger loopen tillhörande **K_id** i listan `unikid`. Denna andra lista innehåller alltså alla unika förekomster av kombinationer.

In [ ]:
unikid = []
behåll = []
for id in range(len(d['K_id'])):
    if d.iloc[id, 0] not in behåll:
        unikid.append(d.iloc[id, 3])
        behåll.append(d.iloc[id, 0])
        

Tills sist skapas tabellen *Kodlistor_stor* genom att endast ha med de rader för vilka **K_id** återfinns i listan `unikid`.

In [ ]:
Kodlistor_stor = d.loc[d['K_id'].isin(unikid)]

Gör om **K_id** till ett index så att den kan agera nyckel i tabellen. 

In [ ]:
Kodlistor_stor.loc[:, 'K_id'] = Kodlistor_stor.index
Kodlistor_stor.set_index('K_id', inplace = True)

***  
skiss:  
1. identifiera varje variabel v s.a. enkät = `w+n` och dubbelkodning = `Nan`. Sådana variabler har två frekvenstabeller efter varandra i radledd.  
2. Låt v förekomma först på rad r och låt j vara antalet koder i kodlistan. Raderna r till r+j-1 ska kopieras och infogas med början på rad r+j  
3. Låt v1 vara v på raderna r till r+j-1 och låt v2 vara v på raderna r+j till r+2j-1. Bägge frekvenstabeller börjar på rad r. Den andra frekvenstabellen ska klippas ut och infogas (i samma kolumn) med start på rad r+j.  
***

### 1.
Läs in *metadata* i en ny `DataFrame` eftersom **Enkät** har blivit ändrad. Identifiera radnummer för de variabler som har **Enkät** = `w+n` samt **Dubbelkodning** = `NaN`.

In [13]:
df1 = pd.read_excel('Metadata.xlsx')

d = df1[(df1.loc[:, 'Enkät'] == 'w+n') & (df1.loc[:, 'Dubbelkodning'].isna() == True)]

### 2.  
Varje kombination av rader med samma variabelnamn kallas för vr och dess första rad kallas r. Notera att avståndet mellan dessa radnummer inte längre är 1. Skapa därför en ny kolumn och lägg raderna där, det blir enklare att loopa över ett "vanligt" index (0, 1, 2, 3, 4, ...) . Välj ut det första radnummret för varje variabel.

In [3]:
d['id'] = d.index

namn = []
r = []

for r in range(len(d.iloc[:, 0])):
    if d.loc[r, 'Namn'] not in namn:
        namn.append(d.loc[r, 'Namn'])
        r.append(d.loc[r, 'id'])

C:\Users\brafo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Låt n vara antalet variabler (vr) som uppfyller villkoret ovan. Den rad variablerna börjar på kallas r. Tabellen *metadata* måste delas upp i n delar (D). Varje D består av D-vr och vr. Varje D har en första rad som kallas s. Ifall två vr följer efter varandra i *metadata* kommer D-vr vara tomt. För vissa D kommer alltså s och r sammanfalla.  
Den första s i *metadata* är såklart `0`. Välj ut resterande s genom att hitta den första rad efter r i vilken variabelnamnet ändras.  

In [4]:
s1 = [0]

for i in r:
    for j in range(i+1, len(df1.loc[:, 'Namn'])):
        if df1.loc[j, 'Namn'] != df1.loc[j-1, 'Namn']:
            s1.append(j)
            break

Nu finns en lista med s som kallas s1. Skapa en lista s2 som är precis som s1 fast utan det första elementet och med ett extra element i slutet. varje par (s1, s2) utgör nu första respektive sista rad i något D. Skär ut samtliga D och lägg i listan stabeller.

In [5]:
s2 = s1[1:]
s2.append(len(df1.iloc[:, 0]))
            
stabeller = []

for i, j in zip(s1, s2):
    stabeller.append(df1.iloc[i:j, :])
    
#stabeller = [df1.iloc[i:j, :] for i,j in zip(s1, s2)]

<span style="color:purple">Här ska jag ersätta värdena i nw-frekvenstabellen med nan för alla element i stabeller.

In [ ]:
for stabell, r in zip(stabeller, r):
    for nwfrek in NONWORKFREK:
        stabell.loc[r:, nwfrek] = np.nan        

På samma sätt utgör (r, s2) första respektive sista rad i vr. Skär ut samtliga vr och lägg i listan rtabeller.

In [6]:
rtabeller = []

for i, j in zip(r, s2):
    rtabeller.append(df1.iloc[i:j, :])
    
#rtabeller = [df1.iloc[i:j, :] for i,j in zip(r, s2)]

<span style="color:purple">Här ska jag ersätta värdena i w-frekvenstabellen med nan för varje element i rtabeller.

In [ ]:
for rtabell in rtabeller:
    for wfrek in WORKFREK:
        rtabell.loc[:, wfrek] = np.nan

Nu ska stabellerna sammanfogas i radledd med sin motsvarighet bland rtabellerna. Spara resultatet i listan srtabeller.

In [7]:
srtabeller = []

for i, j in zip(stabeller, rtabeller):
    srtabeller.append(pd.concat([i, j]))
    
#srtabeller = [pd.concat([i, j]) for i,j in zip(stabeller, rtabeller)]

Sammanfonga nu srtabellerna i radledd och skapa ett nytt index.

In [8]:
d = pd.concat(srtabeller, ignore_index = True)

### 3.  
balblabal


In [12]:
s1 = [1, 2, 3, 4]
s2 = [i-1 for i in s1]
s2

[0, 1, 2, 3]

In [14]:
d.columns

Index(['Namn', 'Tidsserie', 'Beskrivning', 'Enkät', 'Dubbelkodning',
       'Enkättext', 'From', 'Tom', 'Kodlista', 'From.1', 'Tom.1', 'Kod',
       'Text', 'From.2', 'Tom.2', 'Koncept 1', 'Koncept 2', 'Instrument 1',
       'Instrument 2', 'Källa', 'SLOSHW06', 'frek', 'SLOSHW08', 'frek.1',
       'SLOSHW10', 'frek.2', 'SLOSHW12', 'frek.3', 'SLOSHW14', 'frek.4',
       'SLOSHW16', 'frek.5', 'SLOSHW18', 'frek.6', 'SLOSHNW06', 'frek.7',
       'SLOSHNW08', 'frek.8', 'SLOSHNW10', 'frek.9', 'SLOSHNW12', 'frek.10',
       'SLOSHNW14', 'frek.11', 'SLOSHNW16', 'frek.12', 'SLOSHNW18', 'frek.13'],
      dtype='object')

In [17]:
d.loc[40, 'Namn']

'promoted_7'